In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
N_CLASS = 27

In [3]:
MAX_RR = 750 # 60 beats/min => 60 beats/60 s ==> beat/1s ==> 500 samples / beat

In [4]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

In [5]:
from global_vars import labels, normal_class, equivalent_mapping
normal_idx = np.argwhere(labels==int(normal_class))
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

In [16]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

if __name__ == '__main__':
    
    Datas = []
    Header_datas = []
    Classes = []
    Codes = []
    
    dataset_idx = {}
    dataset_data_labels = {}
    dataset_train_idx = {}
    dataset_test_idx = {}
    
    global_idx = 0
    datasets = [1,2,3,4,5,6]
    for dataset in datasets:
        print('Dataset ', dataset)
        # Parse arguments.
        if len(sys.argv) != 3:
            raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

        input_directory = '../NewData/{}/'.format(dataset)
        output_directory = '../Output/'

        # Find files.
        input_files = []
        for f in os.listdir(input_directory):
            if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
                input_files.append(f)

        if not os.path.isdir(output_directory):
            os.mkdir(output_directory)

        classes=get_classes(input_directory,input_files)

        num_files = len(input_files)
        datas = []
        header_datas = []
        dataset_data_labels[dataset] = []
        dataset_idx[dataset] = []
        for i, f in tqdm(enumerate(input_files)):
            #print('    {}/{}...'.format(i+1, num_files), f)
            tmp_input_file = os.path.join(input_directory,f)
            data,header_data = load_challenge_data(tmp_input_file)
            
            codes = get_classes_from_header(header_data)
            
            data_labels = get_scored_class(codes, labels)
            
            datas.append(data[:,1000:7000])
            header_datas.append(header_data)
            Codes.append(codes)
            dataset_data_labels[dataset].append(data_labels)
            dataset_idx[dataset].append(global_idx)
            global_idx += 1

        Datas += datas
        Header_datas += header_datas
        Classes += classes
        
        kf = MultilabelStratifiedKFold(5, random_state=0)
        train_idx, test_idx = next(kf.split(datas, np.array(dataset_data_labels[dataset])))


        dataset_train_idx[dataset] = train_idx +  dataset_idx[dataset][0]
        dataset_test_idx[dataset] = test_idx + dataset_idx[dataset][0]
        
        
        print('Done.')

Dataset  1


/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Done.
Dataset  2



Done.
Dataset  3


/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


...

/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Done.
Dataset  6


/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Done.


In [13]:
import pickle
Q_locs = None
with open('../saved/newData_Q_locs_1000_7000_peakdist100.pkl', 'rb') as Q_locs_file:
    Q_locs = pickle.load(Q_locs_file)

In [20]:
Codes = None
with open('../saved/newData_Codes_1000_7000_peakdist100.pkl', 'rb') as Codes_file:
    Codes = pickle.load(Codes_file)

In [6]:
import pickle
with open('../saved/all_train_idx_stratified.pkl', 'rb') as all_train_idx_file:
    all_train_idx = pickle.load(all_train_idx_file)
with open('../saved/all_test_idx_stratified.pkl', 'rb') as all_test_idx_file:
    all_test_idx = pickle.load(all_test_idx_file)

In [21]:
X_test = []
Data_labels_test = []
Idxes_test = []
for i in tqdm(all_test_idx):
    #print(i)
        
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    for k in ks:
        Data_labels_test.append(get_scored_class(Codes[i], labels))
        X_test.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])
        Idxes_test.append(i)
        

/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [234]:
X_train = []
Data_labels_train = []
Idxes_train = []
Idxes_dict_train = {}
ct = 0
for i in tqdm(all_train_idx):
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
    
    
    Idxes_dict_train[i] = []
    for k in ks:
        Data_labels_train.append(get_scored_class(Codes[i], labels))
        X_train.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])
        Idxes_train.append(i)
        Idxes_dict_train[i].append(ct)
        
    ct += 1

/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# On y va

In [117]:
Signals_test = np.load('../saved/Signals_test_stratified.npy')

In [118]:
import pickle
with open('../saved/Data_labels_test_stratified.pkl', 'rb') as Data_labels_test_file:
    Data_labels_test = pickle.load(Data_labels_test_file)

In [119]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from resnet1d import ECGResNet50

def load_trained_model(model_saved_path):
    model = ECGResNet50(MAX_RR, N_CLASS)
    
    # load saved model
    model.load_state_dict(torch.load(model_saved_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [209]:
model_saved_path = '../saved/ECGResNet50/resnet50_ECGResNet50_minibatch_adam_batch512_0.01_stratified_model.dict'
model = load_trained_model(model_saved_path)

In [121]:
import torch
from torch import nn

In [122]:
Signal_12leads_test= np.transpose(Signals_test, (1,0,2))

In [123]:
from torch.utils.data import Dataset
from torchvision import transforms

import os
class SignalDataset(Dataset):

    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample =(torch.cat([torch.Tensor(np.array([self.signals[channel,idx]]).transpose()) for channel in range(12)], axis=1), 
                  torch.Tensor(self.labels[idx]))

        return sample
    
signal_datasets_test = SignalDataset(Signal_12leads_test, np.array(Data_labels_test)[:,:N_CLASS])


In [124]:
assert np.array(Data_labels_test).shape[1] == N_CLASS

In [125]:
from evaluation.evaluate_12ECG_score import compute_modified_confusion_matrix, compute_challenge_metric

In [126]:
def compute_score(labels, outputs, weights, normal_index=normal_idx):
    num_recordings, num_classes = np.shape(labels)
    # Compute the observed score.
    A = compute_modified_confusion_matrix(labels, outputs)
    observed_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the correct label(s).
    correct_outputs = labels
    A = compute_modified_confusion_matrix(labels, correct_outputs)
    correct_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the normal class.
    inactive_outputs = np.zeros((num_recordings, num_classes), dtype=np.bool)
    inactive_outputs[:, normal_index] = 1
    A = compute_modified_confusion_matrix(labels, inactive_outputs)
    inactive_score = np.nansum(weights * A)

    if correct_score != inactive_score:
        normalized_score = float(observed_score - inactive_score) / float(correct_score - inactive_score)
    else:
        normalized_score = float('nan')

    return normalized_score

In [127]:
from torchvision import datasets, models, transforms
from myeval import agg_y_preds_bags, binary_acc, geometry_loss
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time

st = time.time()
patience = 50
batch_size= 512#65000

saved_dir = '../saved/resnet50/'


testDataset = torch.utils.data.Subset(signal_datasets_test, range(0,len(Signals_test), 1))

testLoader = torch.utils.data.DataLoader(testDataset, batch_size=65000, shuffle = False, pin_memory=True)



In [128]:
np.array(Signals_test).shape

(80312, 12, 750)

In [129]:
names = [get_name(label, Dx_map, Dx_map_unscored) for label in labels]

assert len(labels) == 27

In [210]:
y_tests_tensor = None

output_tests = []
y_tests = []
with torch.no_grad():

    for X_test, y_test in tqdm(testLoader):  
        output_test = model(X_test)
        output_tests.append(output_test)
        y_tests.append(y_test)

    y_tests_tensor = torch.cat(y_tests, axis=0) # ground truth

    output_tests = torch.cat(output_tests, axis=0)
    y_test_preds = torch.sigmoid(output_tests)

#     output_trains = torch.cat(output_trains, axis=0)
#     y_train_preds_max, y_train_preds_mean, _ = agg_y_preds_bags(y_train_preds, bag_size=n_segments)
#     y_test_preds_max, y_test_preds_mean, _ = agg_y_preds_bags(y_test_preds, bag_size=n_segments)
#     _, _, y_trains = agg_y_preds_bags(y_trains, bag_size=n_segments)
#     _, _, y_tests = agg_y_preds_bags(y_tests, bag_size=n_segments)


In [211]:
from global_vars import weights
acc2, fmeasure2, fbeta2, gbeta2 = binary_acc(y_test_preds, y_tests_tensor)
geometry2 = geometry_loss(fbeta2, gbeta2)

score2 = compute_score(np.round(y_test_preds.data.numpy()), np.round(y_tests_tensor.data.numpy()), weights)
output_str = 'End {:.2f} min |\n Valid: Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
    (time.time()-st)/60, acc2, fmeasure2, fbeta2, gbeta2, geometry2, score2)

print(output_str)

#     output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure|Geomotry\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2,geometry2)
#     print(output_string)     
#     with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
#         f.write(output_string)

#     avg_losses_train = np.array(avg_losses_train)
#     avg_losses_test = np.array(avg_losses_test)

#     np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
#     np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)


End 66.98 min |
 Valid: Acc: 0.954, F: 0.194, Fbeta: 0.210, gbeta: 0.099, geo: 0.144, score: -0.188
 


In [212]:
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from global_vars import labels

cf_matrices = multilabel_confusion_matrix(y_tests_tensor.data.numpy(), np.round(y_test_preds.data.numpy()))

In [213]:
for label, cf_matrix in zip(labels, cf_matrices):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')


1st degree av block
[[74669  1383]
 [ 3691   569]]


atrial fibrillation
[[66850  5268]
 [ 4262  3932]]


atrial flutter
[[79514    63]
 [  700    35]]


bradycardia
[[79731    77]
 [  488    16]]


complete right bundle branch block
[[78785   314]
 [  994   219]]


incomplete right bundle branch block
[[77118   396]
 [ 2599   199]]


left anterior fascicular block
[[76199   996]
 [ 2054  1063]]


left axis deviation
[[66033  3937]
 [ 5696  4646]]


left bundle branch block
[[77925   315]
 [ 1125   947]]


low qrs voltages
[[78927   295]
 [  973   117]]


nonspecific intraventricular conduction disorder
[[78660     7]
 [ 1645     0]]


pacing rhythm
[[79765    79]
 [  345   123]]


premature atrial contraction
[[76713     0]
 [ 3587    12]]


premature ventricular contractions
[[79840     0]
 [  472     0]]


prolonged pr interval
[[79755     6]
 [  549     2]]


prolonged qt interval
[[77354   180]
 [ 2740    38]]


qwave abnormal
[[78493    11]
 [ 1807     1]]


right axis deviation


In [214]:
from signal_processing import find_AD

In [215]:
ads = []
for signal, label in tqdm(signal_datasets_test):
    #print(signal.numpy().shape)
    ad = find_AD(signal.numpy().transpose())
    ads.append(ad)

In [216]:
test_pred_labels = np.round(y_test_preds.data.numpy())

In [217]:
test_pred_labels.shape

(80312, 27)

In [218]:
from global_vars import lad_class, rad_class
lad_idx = np.argwhere(labels == int(lad_class)).flatten()
rad_idx = np.argwhere(labels == int(rad_class)).flatten()

In [220]:
for i, ad in enumerate(ads):
    if ad == 'LAD':
        test_pred_labels[i, lad_idx] = 1
        test_pred_labels[i, rad_idx] = 0
    elif ad == 'RAD':
        test_pred_labels[i, rad_idx] = 1
        test_pred_labels[i, lad_idx] = 0
    else:
        test_pred_labels[i, rad_idx] = 0
        test_pred_labels[i, lad_idx] = 0        

In [221]:
cf_matrices2 = multilabel_confusion_matrix(y_tests_tensor.data.numpy(), test_pred_labels)

In [222]:
for label, cf_matrix in zip(labels, cf_matrices2):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')


1st degree av block
[[74669  1383]
 [ 3691   569]]


atrial fibrillation
[[66850  5268]
 [ 4262  3932]]


atrial flutter
[[79514    63]
 [  700    35]]


bradycardia
[[79731    77]
 [  488    16]]


complete right bundle branch block
[[78785   314]
 [  994   219]]


incomplete right bundle branch block
[[77118   396]
 [ 2599   199]]


left anterior fascicular block
[[76199   996]
 [ 2054  1063]]


left axis deviation
[[52561 17409]
 [ 3186  7156]]


left bundle branch block
[[77925   315]
 [ 1125   947]]


low qrs voltages
[[78927   295]
 [  973   117]]


nonspecific intraventricular conduction disorder
[[78660     7]
 [ 1645     0]]


pacing rhythm
[[79765    79]
 [  345   123]]


premature atrial contraction
[[76713     0]
 [ 3587    12]]


premature ventricular contractions
[[79840     0]
 [  472     0]]


prolonged pr interval
[[79755     6]
 [  549     2]]


prolonged qt interval
[[77354   180]
 [ 2740    38]]


qwave abnormal
[[78493    11]
 [ 1807     1]]


right axis deviation


In [223]:
score2_ad = compute_score(np.round(y_tests_tensor.data.numpy()), test_pred_labels, weights)


In [224]:
score2, score2_ad

(-0.1881169960349324, 0.1320815366277788)

In [225]:
def aggr_bag_res(test_pred_labels, Idxes_test):
    assert len(test_pred_labels) == len(Idxes_test)
    uniq = np.unique(Idxes_test, return_counts=True)[1]
    cumsum = np.cumsum(uniq)[:-1]
    res = np.split(test_pred_labels, cumsum)
    agg_res = []
    for group in res:
        agg_res.append((np.sum(group, axis=0)>1).astype(int))
    return np.array(agg_res)
    
agg_res1 = aggr_bag_res(np.round(y_test_preds.data.numpy()), Idxes_test)

agg_res2 = aggr_bag_res(test_pred_labels, Idxes_test)

agg_res_truth = aggr_bag_res(np.round(y_tests_tensor.data.numpy()), Idxes_test)


In [226]:
cf_matrices3 = multilabel_confusion_matrix(agg_res_truth, agg_res1)

In [227]:
for label, cf_matrix in zip(labels, cf_matrices3):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')

1st degree av block
[[7835  305]
 [ 360  118]]


atrial fibrillation
[[6999  924]
 [ 132  563]]


atrial flutter
[[8544   10]
 [  60    4]]


bradycardia
[[8547   14]
 [  52    5]]


complete right bundle branch block
[[8417   64]
 [  89   48]]


incomplete right bundle branch block
[[8220   75]
 [ 279   44]]


left anterior fascicular block
[[8038  219]
 [ 149  212]]


left axis deviation
[[6585  816]
 [ 334  883]]


left bundle branch block
[[8348   62]
 [  60  148]]


low qrs voltages
[[8455   51]
 [  87   25]]


nonspecific intraventricular conduction disorder
[[8419    1]
 [ 198    0]]


pacing rhythm
[[8544   14]
 [  34   26]]


premature atrial contraction
[[8272    0]
 [ 345    1]]


premature ventricular contractions
[[8581    0]
 [  37    0]]


prolonged pr interval
[[8550    0]
 [  67    1]]


prolonged qt interval
[[8283   33]
 [ 295    7]]


qwave abnormal
[[8415    0]
 [ 203    0]]


right axis deviation
[[8528    4]
 [  83    3]]


right bundle branch block
[[7741  396]


In [228]:
score2_agg1 = compute_score(agg_res_truth, agg_res1, weights)
score2_agg2 = compute_score(agg_res_truth, agg_res2, weights)

In [229]:
score2_agg1, score2_agg2

(0.3485566601228477, 0.3362706890886994)